# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK
using AtomsBuilder

## Feeding an AtomsBase AbstractSystem to DFTK

In this example we construct a bulk silicon system using the `bulk` function
from [AtomsBuilder](https://github.com/JuliaMolSim/AtomsBuilder.jl). This function
uses tabulated data to set up a reasonable starting geometry and lattice for bulk silicon.

In [2]:
system = bulk(:Si)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


By default the atoms of an `AbstractSystem` employ the bare Coulomb potential.
To employ pseudpotential models (which is almost always advisable for
plane-wave DFT) one employs the `pseudopotential` keyword argument in
model constructors such as `model_DFT`.
For example we can employ a `PseudoFamily` object
from the [PseudoPotentialData](https://github.com/JuliaMolSim/PseudoPotentialData.jl)
package. See its documentation for more information on the available
pseudopotential families and how to select them.

In [3]:
using PseudoPotentialData  # defines PseudoFamily

pd_lda_family = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
model = model_DFT(system; functionals=LDA(), temperature=1e-3,
                  pseudopotentials=pd_lda_family)

Model(lda_x+lda_c_pw, 3D):
    lattice (in Bohr)    : [0         , 5.13061   , 5.13061   ]
                           [5.13061   , 0         , 5.13061   ]
                           [5.13061   , 5.13061   , 0         ]
    unit cell volume     : 270.11 Bohr³

    atoms                : Si₂
    pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")

    num. electrons       : 8
    spin polarization    : none
    temperature          : 0.001 Ha
    smearing             : DFTK.Smearing.FermiDirac()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(lda_x, lda_c_pw)
                           Entropy()

Alternatively the `pseudopotentials` object also accepts a `Dict{Symbol,String}`,
which provides for each element symbol the filename or identifier
of the pseudopotential to be employed, e.g.

In [4]:
path_to_pspfile = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")[:Si]
model = model_DFT(system; functionals=LDA(), temperature=1e-3,
                  pseudopotentials=Dict(:Si => path_to_pspfile))

Model(lda_x+lda_c_pw, 3D):
    lattice (in Bohr)    : [0         , 5.13061   , 5.13061   ]
                           [5.13061   , 0         , 5.13061   ]
                           [5.13061   , 5.13061   , 0         ]
    unit cell volume     : 270.11 Bohr³

    atoms                : Si₂
    atom potentials      : ElementPsp(:Si, "/home/runner/.julia/artifacts/966fd9cdcd7dbaba6dc2bf43ee50dd81e63e8837/Si.gth")
                           ElementPsp(:Si, "/home/runner/.julia/artifacts/966fd9cdcd7dbaba6dc2bf43ee50dd81e63e8837/Si.gth")

    num. electrons       : 8
    spin polarization    : none
    temperature          : 0.001 Ha
    smearing             : DFTK.Smearing.FermiDirac()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(lda_x, lda_c_pw)
                 

We can then discretise such a model and solve:

In [5]:
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -7.921719844966                   -0.69    5.6    302ms
  2   -7.926133134765       -2.36       -1.22    1.0    183ms
  3   -7.926833191524       -3.15       -2.37    2.0    169ms
  4   -7.926861296547       -4.55       -2.99    3.0    208ms
  5   -7.926861641056       -6.46       -3.37    2.0    179ms
  6   -7.926861669428       -7.55       -3.81    1.2    148ms
  7   -7.926861679592       -7.99       -4.19    1.6    163ms
  8   -7.926861681784       -8.66       -5.11    1.5    163ms
  9   -7.926861681862      -10.11       -5.21    2.4    190ms
 10   -7.926861681871      -11.04       -5.85    1.1    149ms
 11   -7.926861681873      -11.87       -6.60    1.6    164ms
 12   -7.926861681873      -13.39       -7.10    2.2    181ms
 13   -7.926861681873   +  -15.05       -6.92    1.9    174ms
 14   -7.926861681873      -14.57       -6.93    1.2    157ms
 15   -7

If we did not want to use
[AtomsBuilder](https://github.com/JuliaMolSim/AtomsBuilder.jl)
we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
which directly yields an AbstractSystem.

In [6]:
using AtomsIO
system = load_system("Si.extxyz");

Run the LDA calculation:

In [7]:
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(system; pseudopotentials, functionals=LDA(), temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -7.921723310877                   -0.69    5.5    207ms
  2   -7.926135004593       -2.36       -1.22    1.0    193ms
  3   -7.926834986803       -3.15       -2.37    2.0    169ms
  4   -7.926861079598       -4.58       -3.03    2.9    205ms
  5   -7.926861667036       -6.23       -3.49    1.9    193ms
  6   -7.926861677541       -7.98       -4.18    2.1    162ms
  7   -7.926861679691       -8.67       -4.11    2.2    179ms
  8   -7.926861681781       -8.68       -5.09    1.0    146ms
  9   -7.926861681852      -10.15       -5.21    2.5    193ms
 10   -7.926861681872      -10.71       -6.31    1.1    155ms
 11   -7.926861681873      -12.01       -6.96    2.9    205ms
 12   -7.926861681873      -13.80       -7.41    2.2    183ms
 13   -7.926861681873      -14.27       -7.40    1.6    164ms
 14   -7.926861681873   +  -14.45       -8.80    1.0    150ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [8]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(system; pseudopotentials, functionals=LDA(), temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -7.921719913604                   -0.69    5.5    264ms
  2   -7.926135093093       -2.36       -1.22    1.0    133ms
  3   -7.926836813000       -3.15       -2.37    2.0    180ms
  4   -7.926864509931       -4.56       -2.99    3.0    207ms
  5   -7.926865062269       -6.26       -3.38    2.0    170ms
  6   -7.926865082603       -7.69       -3.85    1.4    141ms
  7   -7.926865089592       -8.16       -4.06    1.8    153ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [9]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [10]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, pseudopotentials)
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
